In [1]:
#단순회귀분석
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.formula.api import ols, glm
import scipy as sp

In [2]:
df = pd.read_csv("trafic_data02.csv")

In [3]:
df.head()

,year,month,day,hour,minute,D_N,DayofWeek,Death_cnt,Casualties_cnt,Serious_cnt,...,Road_Type_L,Road_Type,Dang1_L,Dang1,Dang2_L,Dang2,X_UTMK,Y_UTMK,Longitude,Latitude
0,2017,6,30,23,14,0,0,1,1,0,...,5,8,5,9,2,4,967633,1937456,127.134130,37.435704
1,2017,6,30,21,45,0,0,1,4,2,...,5,8,5,9,5,9,966456,1696405,127.131220,35.262534
2,2017,6,30,21,50,0,0,1,1,0,...,5,8,5,9,2,4,900057,1865697,126.379922,36.784142
3,2017,6,30,19,10,0,0,1,1,0,...,2,4,5,9,10,14,1095230,1764891,128.554945,35.875973
4,2017,6,30,17,2,1,0,1,2,1,...,5,2,8,12,13,18,947576,1946355,126.906786,37.514993


In [4]:
#test에 필요한 열들 추출
sort_idx = list(df.columns[5:16])+['Violation_Law','Road_Type_L','Road_Type','Dang1_L','Dang2_L']

In [5]:
sort_idx

['D_N',
 'DayofWeek',
 'Death_cnt',
 'Casualties_cnt',
 'Serious_cnt',
 'Slightly_cnt',
 'Injury_Report_cnt',
 'City_Do',
 'City_Gun_Gu',
 'Type_Large_Group',
 'Type_Middle_Gruop',
 'Violation_Law',
 'Road_Type_L',
 'Road_Type',
 'Dang1_L',
 'Dang2_L']

In [6]:
#새로운 df 생성
train_df = df[sort_idx]

In [7]:
# 각변수간의 상관분석
train_df.corr()

,D_N,DayofWeek,Death_cnt,Casualties_cnt,Serious_cnt,Slightly_cnt,Injury_Report_cnt,City_Do,City_Gun_Gu,Type_Large_Group,Type_Middle_Gruop,Violation_Law,Road_Type_L,Road_Type,Dang1_L,Dang2_L
D_N,1.000000,-0.018668,0.003208,0.047489,0.037285,0.045834,0.008982,0.023498,0.014418,0.122169,-0.017093,0.028382,-0.052375,-0.051025,0.117933,0.149889
DayofWeek,-0.018668,1.000000,0.024661,0.002935,0.004822,0.000477,-0.010757,0.004395,-0.004037,0.019656,-0.006307,0.000309,0.016012,0.007893,-0.037285,-0.005559
Death_cnt,0.003208,0.024661,1.000000,0.310176,0.199485,0.111527,0.085023,-0.000667,0.013394,0.072021,-0.027897,0.018744,0.019412,-0.001114,-0.019437,0.052560
Casualties_cnt,0.047489,0.002935,0.310176,1.000000,0.770968,0.808983,0.433733,-0.003369,0.002391,0.103082,0.013652,0.043096,0.003370,-0.025811,-0.011787,0.109677
Serious_cnt,0.037285,0.004822,0.199485,0.770968,1.000000,0.366464,0.108955,-0.002371,0.005417,0.113603,-0.006448,0.048766,0.005674,-0.022904,-0.017715,0.103770
Slightly_cnt,0.045834,0.000477,0.111527,0.808983,0.366464,1.000000,0.184892,0.005345,-0.001727,0.056548,0.035013,0.024152,-0.009419,-0.026493,0.002763,0.080431
Injury_Report_cnt,0.008982,-0.010757,0.085023,0.433733,0.108955,0.184892,1.000000,-0.019138,-0.004058,0.016508,0.003187,0.008652,0.012605,0.003295,-0.007004,0.019152
City_Do,0.023498,0.004395,-0.000667,-0.003369,-0.002371,0.005345,-0.019138,1.000000,0.010810,0.018467,-0.004342,-0.004496,-0.018695,-0.026773,0.013786,0.010491
City_Gun_Gu,0.014418,-0.004037,0.013394,0.002391,0.005417,-0.001727,-0.004058,0.010810,1.000000,0.037211,-0.011165,0.018691,0.012324,-0.006593,-0.003088,0.015323
Type_Large_Group,0.122169,0.019656,0.072021,0.103082,0.113603,0.056548,0.016508,0.018467,0.037211,1.000000,-0.416854,0.152803,0.082822,-0.053884,0.046527,0.596065


In [8]:
# 사용할 열들
train_df.columns=['D_N',
 'DayofWeek',
 'Death_cnt',
 'Casualties_cnt',
 'Serious_cnt',
 'Slightly_cnt',
 'Injury_Report_cnt',
 'City_Do',
 'City_Gun_Gu',
 'Type_Large_Group',
 'Type_Middle_Group',
 'Violation_Law',
 'Road_Type_L',
 'Road_Type',
 'Dang1_L',
 'Dang2_L']

In [9]:
train_df.columns

Index(['D_N', 'DayofWeek', 'Death_cnt', 'Casualties_cnt', 'Serious_cnt',
       'Slightly_cnt', 'Injury_Report_cnt', 'City_Do', 'City_Gun_Gu',
       'Type_Large_Group', 'Type_Middle_Group', 'Violation_Law', 'Road_Type_L',
       'Road_Type', 'Dang1_L', 'Dang2_L'],
      dtype='object')

In [10]:
# 단순회귀식 작성
# 다른 더미변수를 통한 사망자수 예측을 위한 회귀식
my_fml = 'Death_cnt ~ D_N + DayofWeek + City_Do + City_Gun_Gu + Type_Large_Group + Type_Middle_Group + Violation_Law + Road_Type_L + Road_Type + Dang1_L + Dang2_L'
lm = ols(my_fml, data = train_df).fit()
lm.summary()

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Death_cnt   R-squared:                       0.007
Model:                            OLS   Adj. R-squared:                  0.007
Method:                 Least Squares   F-statistic:                     16.01
Date:                Thu, 26 Jul 2018   Prob (F-statistic):           8.78e-32
Time:                        13:45:39   Log-Likelihood:                -977.53
No. Observations:               25037   AIC:                             1979.
Df Residuals:                   25025   BIC:                             2077.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
=====================================================================================
                        coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept             0.9870      0.011     92.632      0.000       0.966       1.008
D_N                  -0.0016      0.003     -0.495      0.620      -0.008       0.005
DayofWeek             0.0028      0.001      3.553      0.000       0.001       0.004
City_Do           -8.021e-05      0.000     -0.273      0.785      -0.001       0.000
City_Gun_Gu        4.338e-05   2.62e-05      1.653      0.098   -8.06e-06    9.48e-05
Type_Large_Group      0.0193      0.003      6.562      0.000       0.014       0.025
Type_Middle_Group     0.0001      0.000      0.464      0.643      -0.000       0.001
Violation_Law         0.0003      0.001      0.563      0.573      -0.001       0.002
Road_Type_L           0.0061      0.002      2.968      0.003       0.002       0.010
Road_Type            -0.0021      0.001     -2.124      0.034      -0.004      -0.000
Dang1_L              -0.0022      0.001     -3.696      0.000      -0.003      -0.001
Dang2_L               0.0013      0.001      2.394      0.017       0.000       0.002
==============================================================================
Omnibus:                    38550.786   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         26613714.792
Skew:                           9.659   Prob(JB):                         0.00
Kurtosis:                     161.550   Cond. No.                         825.
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
"""

In [11]:
# p 값이 0.05보다 큰 변수 제거 후에 다시 회귀식 작성
my_fml = 'Death_cnt ~ DayofWeek + City_Do * City_Gun_Gu + Type_Large_Group * Type_Middle_Group + Road_Type_L + Road_Type + Dang1_L + Dang2_L'
lm = ols(my_fml, data = train_df).fit()
lm.summary()

#단순 선형 회귀식이 무의미하다는 것을 알 수 있음
#R-Sq 값이 너무 낮으므로 해당 회귀선으로 설명할 수 있는 개체가 0퍼임

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:              Death_cnt   R-squared:                       0.008
Model:                            OLS   Adj. R-squared:                  0.008
Method:                 Least Squares   F-statistic:                     19.29
Date:                Thu, 26 Jul 2018   Prob (F-statistic):           3.28e-39
Time:                        13:45:39   Log-Likelihood:                -959.59
No. Observations:               25037   AIC:                             1943.
Df Residuals:                   25025   BIC:                             2041.
Df Model:                          11                                         
Covariance Type:            nonrobust                                         
======================================================================================================
                                         coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------------------
Intercept                              1.0177      0.011     92.293      0.000       0.996       1.039
DayofWeek                              0.0029      0.001      3.627      0.000       0.001       0.004
City_Do                               -0.0006      0.001     -1.001      0.317      -0.002       0.001
City_Gun_Gu                         1.244e-05   4.08e-05      0.305      0.760   -6.75e-05    9.24e-05
City_Do:City_Gun_Gu                 4.673e-06   4.87e-06      0.960      0.337   -4.87e-06    1.42e-05
Type_Large_Group                       0.0083      0.003      2.410      0.016       0.002       0.015
Type_Middle_Group                     -0.0036      0.001     -5.265      0.000      -0.005      -0.002
Type_Large_Group:Type_Middle_Group     0.0024      0.000      5.960      0.000       0.002       0.003
Road_Type_L                            0.0056      0.002      2.787      0.005       0.002       0.010
Road_Type                             -0.0015      0.001     -1.512      0.131      -0.003       0.000
Dang1_L                               -0.0024      0.001     -4.075      0.000      -0.004      -0.001
Dang2_L                             2.897e-05      0.001      0.050      0.960      -0.001       0.001
==============================================================================
Omnibus:                    38529.678   Durbin-Watson:                   2.011
Prob(Omnibus):                  0.000   Jarque-Bera (JB):         26625557.458
Skew:                           9.647   Prob(JB):                         0.00
Kurtosis:                     161.589   Cond. No.                     7.74e+03
==============================================================================

Warnings:
[1] Standard Errors assume that the covariance matrix of the errors is correctly specified.
[2] The condition number is large, 7.74e+03. This might indicate that there are
strong multicollinearity or other numerical problems.
"""

In [13]:
#포아송회귀분석
%pylab inline
import pylab as pb
from scipy import stats
import scipy as sp

Populating the interactive namespace from numpy and matplotlib


In [24]:
from statsmodels.genmod.families import Poisson
from statsmodels.genmod.cov_struct import (Exchangeable,
    Independence,Autoregressive)
from statsmodels.genmod.generalized_estimating_equations import GEE

In [36]:
#포아송 분포로 식 분석

model = GEE.from_formula(formula = my_fml,groups=train_df['Death_cnt'], data=train_df, cov_struct=Independence(), family=Poisson())


In [39]:
my_fml = 'Death_cnt ~ D_N + DayofWeek + City_Do + City_Gun_Gu + Type_Large_Group + Type_Middle_Group + Violation_Law + Road_Type_L + Road_Type + Dang1_L + Dang2_L'

In [40]:
result = model.fit()

In [41]:
result.summary()

# p 값이 다 무의미함 (0.05 이하로 되는 값이 하나도 없음)

<class 'statsmodels.iolib.summary.Summary'>
"""
                               GEE Regression Results                              
===================================================================================
Dep. Variable:                   Death_cnt   No. Observations:                25037
Model:                                 GEE   No. clusters:                        8
Method:                        Generalized   Min. cluster size:                   1
                      Estimating Equations   Max. cluster size:               24206
Family:                            Poisson   Mean cluster size:              3129.6
Dependence structure:         Independence   Num. iterations:                     5
Date:                     Thu, 26 Jul 2018   Scale:                           1.000
Covariance type:                    robust   Time:                         15:21:21
=====================================================================================
                        coef    std err          z      P>|z|      [0.025      0.975]
-------------------------------------------------------------------------------------
Intercept            -0.0119      0.015     -0.794      0.427      -0.041       0.017
D_N                  -0.0016      0.006     -0.268      0.788      -0.013       0.010
DayofWeek             0.0027      0.003      0.927      0.354      -0.003       0.008
City_Do            -7.67e-05      0.000     -0.242      0.809      -0.001       0.001
City_Gun_Gu        4.167e-05   4.21e-05      0.989      0.323   -4.09e-05       0.000
Type_Large_Group      0.0184      0.020      0.930      0.353      -0.020       0.057
Type_Middle_Group     0.0001      0.000      0.749      0.454      -0.000       0.000
Violation_Law         0.0003      0.000      0.863      0.388      -0.000       0.001
Road_Type_L           0.0059      0.006      0.908      0.364      -0.007       0.018
Road_Type            -0.0020      0.002     -0.842      0.400      -0.007       0.003
Dang1_L              -0.0021      0.002     -0.948      0.343      -0.007       0.002
Dang2_L               0.0013      0.002      0.832      0.405      -0.002       0.004
==============================================================================
Skew:                          9.6585   Kurtosis:                     158.5483
Centered skew:                -0.0761   Centered kurtosis:             -0.8388
==============================================================================
"""